##  Tested on these samples 32-bit

### 0873b7a5cfae17a6dfebe6afde535a186b08d76b4b8ef56a129459c56f016729
### cc4ce27b042213058ffb13a5078b681dc99e516fb2861b8b3637a25681fd15ec
### a398e940847ee51fa7fed05b1c6b38a47c1668c14616a0f39f56fd314fe92ad8

> In order to get somewhere, we need to know where we are.

## ref: https://github.com/n1ght-w0lf/MalwareAnalysis/tree/master/GCleaner

In [126]:
import base64
from Crypto.Cipher import ARC4

# decode strings from base64 then decrypt from RC4
def decryption_func(obfuscated_str, key):
    try:
        decoded_data = base64.b64decode(obfuscated_str)
        cipher = ARC4.new(key)
        decrypted_data = cipher.decrypt(decoded_data)
        # Convert the decrypted bytes to a string
        return decrypted_data.decode()
    except Exception as e:
        return None


In [127]:
import pefile
import re
from capstone import *
from capstone.x86 import *
pattern1 = rb'\xC7\x05\x84\x82\x61\x00\x54\x4D\x41\x00'  # To get rc4 key 

pe = pefile.PE(r"") # Put your file path

# Initialize Capstone
md = Cs(CS_ARCH_X86, CS_MODE_32)
md.skipdata = True
md.detail = True


In [128]:
# Find .text section
insns = []
text_section = pe.sections[0]
text_section_addr = pe.OPTIONAL_HEADER.ImageBase + text_section.VirtualAddress

# Disassemble .text section
for insn in md.disasm(text_section.get_data(), text_section_addr):
    insns.append(insn)
    # print(insn)

# Get address of rc4 key from the .text section
for i in range(len(insns)):
    if re.match(pattern1, insns[i].bytes):

        insn = insns[i]
        if (insn.mnemonic == "mov" and
            insn.operands[0].type == X86_OP_MEM and
            insn.operands[1].type == X86_OP_IMM):
            # Get the address of the RC4 key string from .data section
            rc4_key_addr = insn.operands[1].imm
            print(hex(rc4_key_addr))

0x414d54


In [129]:
# Get rc4_key from .data
rdata_data_list = []
rdata_section = pe.sections[1]
rdata_section_addr =pe.OPTIONAL_HEADER.ImageBase + rdata_section.VirtualAddress
# print(hex(rdata_section_addr))

# Calculate the offset within the section
offset_within_section = rc4_key_addr - rdata_section_addr

# Get the raw data of the section
section_data = rdata_section.get_data()

# Retrieve 25 bytes of data starting from the specified address
rc4_key = section_data[offset_within_section:offset_within_section + 25]
# rc4_key = rc4_key.decode()
print(rc4_key)


b'8106833130137977202684225'


In [133]:
# get all strings from .data section
data = rdata_section.get_data()

# Split data into strings using null-termination (00)
strings = []
current_string = b""

for byte in data:
    if byte != 0:
        current_string += bytes([byte])
    elif current_string:
        decoded_string = current_string.decode('utf-8', errors='replace')
        if all(ord(char) < 128 for char in decoded_string):
            strings.append(decoded_string)
        current_string = b""

# Print extracted ASCII strings
for string in strings:
    print(string)


v
F
F
F@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@>@@@?456789:;<=@@@@@@@
	
@@@@@@ !"#$%&'()*+,-./0123@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+/
%d
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@>@@@?456789:;<=@@@@@@@
	
@@@@@@ !"#$%&'()*+,-./0123@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+/
i
m
a
g
e
/
j
p
e
g
%s%s
 / 
A
B
C
D
E
F
G
H
I
J
K
L
M
N
O
P
Q
R
S
T
U
V
W
X
Y
Z
a
b
c
d
e
f
g
h
i
j
k
l
m
n
o
p
q
r
s
t
u
v
w
x
y
z
0
1
2
3
4
5
6
7
8
9
+
/
=
%d
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@>@@@?456789:;<=@@@@@@@
	
@@@@@@ !"#$%&'()*+,-./0123@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

In [134]:
# decrypt strings and find ip and 
import re

for idx, string in enumerate(strings, start=1):
    decrypted_string = decryption_func(string, rc4_key)
    if decrypted_string is not None:
        if decrypted_string != '':
            # print(decrypted_string)
             # Find IP addresses using the regex pattern
            ip_addresses = re.findall(r"\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}", decrypted_string)
            # Find file paths using the regex pattern
            file_paths = re.findall(r"/[\w/]+\.php", decrypted_string)
            
            if ip_addresses:
                print(f"IP addresses in String {idx}: {', '.join(ip_addresses)}")
            
            if file_paths:
                print(f"File paths in String {idx}: {', '.join(file_paths)}")

IP addresses in String 413: 94.131.107.238
File paths in String 414: /3aa13fff14e398a1.php
